<div style="display:flex;flex-direction:row;justify-content: space-evenly;">
<img src="tec.png" style="width:320px;height:197px;"/>
<img src="itcolima.svg" style="width:197px"/>
</div>
<div style="display:flex;flex-direction:column;gap:20px;text-align:center"></div>

<h1 align="center">Tecnológico Nacional de México
campus Colima</h1>
<h2 align="center">Maestría en Sistemas
Computacionales</h2><br></br>
<h3 align="center">Primitivas de Sincronización</h3>
<h3 align="center">Ejercicio Práctico:Primitivas de Sincronización</h3><br></br>
<h4 align="center">Docente:</h4>
<h4 align="center">D. en C. Patricia Figueroa Millan </h4><br></br>
<h4 align="center">Autor:</h4>
<h4 align="center">Ing. Gilberto Rene Martinez Gutierrez G2146013</h4><br></br>
<h4 align="center"></h4>
<h4 align="center">Villa de Álvarez, Colima, México</h4>
<h4 align="center">08 de noviembre de 2022</h4>

<h2 align="rigth" >Objetivo</h2>

<h4 align="rigth" >Desarrollar un ejercicio sobre los usos e implementacion de las primitivas de sincronizacion</h4>

<h2 align="rigth" >Metodologia</h2>
<h4 align="justify" >Para la resolucion del problema se realizo un analisis previo sobre el funcionamiento de las primitivas de sincronizacion donde se recurrio a la investigacion en medios especiales, el contenido de este material facilito el entendimiento, ademas de permitir analisar unos ejemplos de su funcionamiento </h4>

<h2 align="rigth" >Look </h2>

<h4 align="rigth" >A primitive lock is in one of two states, “locked” or “unlocked”. It is created in the unlocked state. It has two basic methods, acquire() and release(). When the state is unlocked, acquire() changes the state to locked and returns immediately. When the state is locked, acquire() blocks until a call to release() in another thread changes it to unlocked, then the acquire() call resets it to locked and returns. </h4>

In [3]:
import threading
import asyncio
Lock=threading.Lock()
ITERS = 1000000
x = [0]

#asyncio functions
def worker():
    for _ in range(ITERS):       
        with Lock:
            x[0] += 1

def main():
    x[0] = 0  # you may use `global x` instead of this list trick too
    t1 = threading.Thread(target=worker)
    t2 = threading.Thread(target=worker)

    t1.start()
    t2.start()
    t1.join()
    t2.join()

for i in range(5):
    main()
    print(f'iteration {i}. expected x = {ITERS*2}, got {x[0]}')

iteration 0. expected x = 2000000, got 2000000
iteration 1. expected x = 2000000, got 2000000
iteration 2. expected x = 2000000, got 2000000
iteration 3. expected x = 2000000, got 2000000
iteration 4. expected x = 2000000, got 2000000


<h2 align="rigth" >Event </h2>

<h4 align="rigth" >This is one of the simplest mechanisms for communication between threads: one thread signals an event and other threads wait for it.

An event object manages an internal flag that can be set to true with the set() method and reset to false with the clear() method. The wait() method blocks until the flag is true.</h4>

In [ ]:
from time import sleep
from random import random
from threading import Thread
from threading import Event
 
# target task function
def task(event, number):
    # wait for the event to be set
    event.wait()
    # begin processing
    print("Se inicio el hilo")



# create a shared event object
event = Event()
# create a suite of threads
for i in range(5):
    thread = Thread(target=task, args=(event, i))
    thread.start()
    
# block for a moment
print('Main thread blocking... ')

sleep(10)
# start processing in all threads
event.set()
# wait for all the threads to finish...



<h2 align="rigth" >Condition Objects </h2>

<h4 align="rigth" >A condition variable is always associated with some kind of lock; this can be passed in or one will be created by default. Passing one in is useful when several condition variables must share the same lock. The lock is part of the condition object: you don’t have to track it separately.

A condition variable obeys the context management protocol: using the with statement acquires the associated lock for the duration of the enclosed block. The acquire() and release() methods also call the corresponding methods of the associated lock.</h4>

In [ ]:
import threading
import time
from random import randint

class SomeItem:
  # init method
  def __init__(self):
    # initialize empty list
    self.list = []
  
  # add to list method for producer
  def produce(self, item):
    print("Adding item to list...")
    self.list.append(item)
    
  # remove item from list method for consumer
  def consume(self):
    print("consuming item from list...")
    item = self.list[0]
    print("Item consumed: ", item)
    self.list.remove(item)

def producer(si, cond):
    r = randint(1,5)
    # creating random number of items
    for i in range(1, r):
      print("working on item creation, it will take: " + str(i) + " seconds")
      time.sleep(i)
      print("acquiring lock...")
      cond.acquire()
      try:
        si.produce(i)
        cond.notify()
      finally:
        cond.release()
      
      
def consumer(si, cond):
    cond.acquire()
    while True:
      try:
        si.consume()
      except:   
        print("No item to consume...")
        # wait with a maximum timeout of 10 sec
        val = cond.wait(10)
        if val:
          print("notification received about item production...")
          continue
        else:
          print("waiting timeout...")
          break
        
    cond.release()
    
if __name__=='__main__':
  # condition object
  cond = threading.Condition()
  # SomeItem object
  si = SomeItem()
  # producer thread
  p = threading.Thread(target=producer, args=(si,cond,))
  p.start()
  # consumer thread
  c = threading.Thread(target=consumer, args=(si,cond,))
  c.start()

  #print('Waiting for producer and consumer threads...')
  p.join()
  c.join()
  print("Done")

<h2 align="rigth" >Semaphore Objects </h2>

<h4 align="rigth" >This is one of the oldest synchronization primitives in the history of computer science, invented by the early Dutch computer scientist Edsger W. Dijkstra (he used the names P() and V() instead of acquire() and release()).

A semaphore manages an internal counter which is decremented by each acquire() call and incremented by each release() call. The counter can never go below zero; when acquire() finds that it is zero, it blocks, waiting until some other thread calls release().</h4>

In [ ]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)


class ThreadPool(object):
    def __init__(self):
        super(ThreadPool, self).__init__()
        self.active = []
        self.lock = threading.Lock()
    def makeActive(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug('Running: %s', self.active)
    def makeInactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug('Running: %s', self.active)


def f(s, pool):
    logging.debug('Waiting to join the pool')
    with s:
        name = threading.currentThread().getName()
        pool.makeActive(name)
        time.sleep(0.5)
        pool.makeInactive(name)


if __name__ == '__main__':
    pool = ThreadPool()
    s = threading.Semaphore(3)
    for i in range(5):
        t = threading.Thread(target=f, name='thread_'+str(i), args=(s, pool))
        t.start()

<h2 align="rigth" >Barrier Objects </h2>

<h4 align="rigth" >This class provides a simple synchronization primitive for use by a fixed number of threads that need to wait for each other. Each of the threads tries to pass the barrier by calling the wait() method and will block until all of the threads have made their wait() calls. At this point, the threads are released simultaneously.

The barrier can be reused any number of times for the same number of threads.

As an example, here is a simple way to synchronize a client and server thread:</h4>

In [ ]:
import threading
import time

def start_server():
  # starting server
  print("starting the server...")
  # do some startup work
  time.sleep(2)
  

def server(b):
    start_server()
    b.wait()
    print("Server is ready.")

def client(b):
    print("waiting for server getting ready...")
    b.wait()
    print("sending request to server...")

if __name__=='__main__':
  
  b = threading.Barrier(2, timeout=5)
  # server thread
  s = threading.Thread(target=server, args=(b,))
  s.start()
  # client thread
  c = threading.Thread(target=client, args=(b,))
  c.start()

  s.join()
  c.join()
  print("Done")

<h2 align="rigth" >Conclusiones</h2>

<h4 align="justify" >Gracias al desarrolo de esta actividad se logro comprender el funcionamiento de las primitivas de sincronizacion, a traves de este se comprendio la estructura con la cual se deben diseñar sistemas multihilo</h4>